<a href="https://colab.research.google.com/github/thewildofficial/pytorch-notebooks/blob/main/Space_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import torch
from torch import nn

train_df = pd.read_csv("https://raw.githubusercontent.com/pirosbogar/spaceship-titanic/main/train.csv")
test_df = pd.read_csv("https://raw.githubusercontent.com/pirosbogar/spaceship-titanic/main/test.csv")
submitted_vals = pd.read_csv("https://raw.githubusercontent.com/pirosbogar/spaceship-titanic/main/submission.csv")

test_df["Transported"] = submitted_vals["Transported"]


In [45]:
test_df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,True
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,False
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,True
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,True
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter,True
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron,False
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,True
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,True


In [46]:
from sklearn.preprocessing import MinMaxScaler

features = ["HomePlanet","CryoSleep","Age","VIP","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","Transported"]
for i, df in enumerate([train_df, test_df]):
  df["HomePlanet"] = df["HomePlanet"].replace({"Europa": 0, "Earth": 1, "Mars": 2})
  df = df[features].astype(float).dropna(axis=0)
  scaler = MinMaxScaler()
  normalize_columns = ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]
  df[normalize_columns] = scaler.fit_transform(df[normalize_columns])
  if i == 0:
    train_df = df
  else:
    test_df = df

In [47]:
train_df

,HomePlanet,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0.0,0.0,39.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1.0,0.0,24.0,0.0,0.007608,0.000302,0.001064,0.024500,0.002164,1.0
2,0.0,0.0,58.0,1.0,0.003001,0.119948,0.000000,0.299670,0.002410,0.0
3,0.0,0.0,33.0,0.0,0.000000,0.043035,0.015793,0.148563,0.009491,0.0
4,1.0,0.0,16.0,0.0,0.021149,0.002348,0.006428,0.025214,0.000098,1.0
...,...,...,...,...,...,...,...,...,...,...
8688,0.0,0.0,41.0,1.0,0.000000,0.228726,0.000000,0.073322,0.003639,0.0
8689,1.0,1.0,18.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8690,1.0,0.0,26.0,0.0,0.000000,0.000000,0.079687,0.000045,0.000000,1.0
8691,0.0,0.0,32.0,0.0,0.000000,0.035186,0.000000,0.015753,0.159077,0.0


In [48]:
X_train = train_df.drop("Transported",axis=1)
y_train = train_df["Transported"]

X_test = test_df.drop("Transported",axis=1)
y_test = test_df["Transported"]

In [49]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [50]:
X_train = torch.tensor(X_train.values).to(device).to(torch.float32)
X_test = torch.tensor(X_test.values).to(device).to(torch.float32)
y_train = torch.tensor(y_train.values).to(device).to(torch.float32)
y_test = torch.tensor(y_test.values).to(device).to(torch.float32)

In [51]:
X_train.shape,y_train.shape

(torch.Size([7074, 9]), torch.Size([7074]))

In [52]:
in_size = len(features) - 1
hidden_size = 64

model = nn.Sequential(
    nn.Linear(in_size,hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size,hidden_size),
    nn.ELU(),
    nn.Linear(hidden_size,hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size,hidden_size),
    nn.Tanh(),
    nn.Linear(hidden_size,1),
    nn.Sigmoid()
)
model.to(device)

Sequential(
  (0): Linear(in_features=9, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ELU(alpha=1.0)
  (4): Linear(in_features=64, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=64, bias=True)
  (7): Tanh()
  (8): Linear(in_features=64, out_features=1, bias=True)
  (9): Sigmoid()
)

In [53]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(lr=0.01,params = model.parameters())



In [54]:
def accuracy_fn(true_values,predicted_values):
  correct = torch.eq(true_values,predicted_values).sum().item()
  acc = (correct/len(true_values)) * 100
  return acc

In [55]:
epochs = 1000
for epoch in range(epochs):
  y_preds = model.forward(X_train).squeeze()
  loss = loss_fn(y_preds, y_train)
  accuracy = accuracy_fn(y_train,y_preds.round())
  

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  model.eval()
  with torch.inference_mode():
    y_preds = model.forward(X_test).squeeze()
    test_acc = accuracy_fn(y_test,y_preds.round())
  if test_acc  >= 90:
    break
  print(f"Epoch:[{epoch+1}/{epochs}]| Loss: {loss}| Train Accuracy: {accuracy}% | Test Accuracy: {test_acc}%")

Epoch:[1/1000]| Loss: 0.7036399245262146| Train Accuracy: 50.26858919988692% | Test Accuracy: 47.86081003993154%
Epoch:[2/1000]| Loss: 0.8361844420433044| Train Accuracy: 49.73141080011309% | Test Accuracy: 52.13918996006846%
Epoch:[3/1000]| Loss: 0.7800620198249817| Train Accuracy: 50.26858919988692% | Test Accuracy: 51.3405590416429%
Epoch:[4/1000]| Loss: 0.6920421123504639| Train Accuracy: 51.08849307322589% | Test Accuracy: 51.59726183685112%
Epoch:[5/1000]| Loss: 0.709585964679718| Train Accuracy: 52.756573367260394% | Test Accuracy: 51.968054763262984%
Epoch:[6/1000]| Loss: 0.6929411292076111| Train Accuracy: 53.18066157760815% | Test Accuracy: 52.13918996006846%
Epoch:[7/1000]| Loss: 0.6869352459907532| Train Accuracy: 50.26858919988692% | Test Accuracy: 52.13918996006846%
Epoch:[8/1000]| Loss: 0.6906390190124512| Train Accuracy: 50.26858919988692% | Test Accuracy: 52.13918996006846%
Epoch:[9/1000]| Loss: 0.684993326663971| Train Accuracy: 50.26858919988692% | Test Accuracy: 52.

In [57]:
test_acc

90.13120365088419